In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
import seaborn as sns
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import string
# import transformers
# nltk.download('punkt')
# nltk.download('stopwords')
stop = stopwords.words('english')
import re
from sklearn import feature_extraction, model_selection, naive_bayes, pipeline, manifold, preprocessing, metrics
from lime import lime_text
import gensim
import gensim.downloader as gensim_api
from tensorflow.keras import models, layers, preprocessing as kprocessing
from tensorflow.keras import backend as K

In [2]:
all_wines = pd.read_csv('Data/winemag-data-130k-v2.csv')
all_wines.head()

,Unnamed: 0,country,description,designation,points,price,province,region_1,region_2,taster_name,taster_twitter_handle,title,variety,winery
0,0,Italy,"Aromas include tropical fruit, broom, brimston...",Vulkà Bianco,87,NaN,Sicily & Sardinia,Etna,NaN,Kerin O’Keefe,@kerinokeefe,Nicosia 2013 Vulkà Bianco (Etna),White Blend,Nicosia
1,1,Portugal,"This is ripe and fruity, a wine that is smooth...",Avidagos,87,15.0,Douro,NaN,NaN,Roger Voss,@vossroger,Quinta dos Avidagos 2011 Avidagos Red (Douro),Portuguese Red,Quinta dos Avidagos
2,2,US,"Tart and snappy, the flavors of lime flesh and...",NaN,87,14.0,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,@paulgwine,Rainstorm 2013 Pinot Gris (Willamette Valley),Pinot Gris,Rainstorm
3,3,US,"Pineapple rind, lemon pith and orange blossom ...",Reserve Late Harvest,87,13.0,Michigan,Lake Michigan Shore,NaN,Alexander Peartree,NaN,St. Julian 2013 Reserve Late Harvest Riesling ...,Riesling,St. Julian
4,4,US,"Much like the regular bottling from 2012, this...",Vintner's Reserve Wild Child Block,87,65.0,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,@paulgwine,Sweet Cheeks 2012 Vintner's Reserve Wild Child...,Pinot Noir,Sweet Cheeks


In [3]:
clean_wines = all_wines.drop(columns=['Unnamed: 0', 'taster_twitter_handle', 'region_2'])
clean_wines = clean_wines.dropna()
clean_wines.head()

,country,description,designation,points,price,province,region_1,taster_name,title,variety,winery
3,US,"Pineapple rind, lemon pith and orange blossom ...",Reserve Late Harvest,87,13.0,Michigan,Lake Michigan Shore,Alexander Peartree,St. Julian 2013 Reserve Late Harvest Riesling ...,Riesling,St. Julian
4,US,"Much like the regular bottling from 2012, this...",Vintner's Reserve Wild Child Block,87,65.0,Oregon,Willamette Valley,Paul Gregutt,Sweet Cheeks 2012 Vintner's Reserve Wild Child...,Pinot Noir,Sweet Cheeks
5,Spain,Blackberry and raspberry aromas show a typical...,Ars In Vitro,87,15.0,Northern Spain,Navarra,Michael Schachner,Tandem 2011 Ars In Vitro Tempranillo-Merlot (N...,Tempranillo-Merlot,Tandem
6,Italy,"Here's a bright, informal red that opens with ...",Belsito,87,16.0,Sicily & Sardinia,Vittoria,Kerin O’Keefe,Terre di Giurfo 2013 Belsito Frappato (Vittoria),Frappato,Terre di Giurfo
9,France,This has great depth of flavor with its fresh ...,Les Natures,87,27.0,Alsace,Alsace,Roger Voss,Jean-Baptiste Adam 2012 Les Natures Pinot Gris...,Pinot Gris,Jean-Baptiste Adam


In [4]:
clean_wines['variety'] = clean_wines['variety'].replace(['Weissburgunder'], 'Chardonnay')
clean_wines['variety'] = clean_wines['variety'].replace(['Spatburgunder'], 'Pinot Noir')
clean_wines['variety'] = clean_wines['variety'].replace(['Grauburgunder'], 'Pinot Gris')
clean_wines['variety'] = clean_wines['variety'].replace(['Garnacha'], 'Grenache')
clean_wines['variety'] = clean_wines['variety'].replace(['Pinot Nero'], 'Pinot Noir')
clean_wines['variety'] = clean_wines['variety'].replace(['Alvarinho'], 'Albarino')

In [5]:
clean_wines['vintage'] = clean_wines['title'].str.extract('(\d\d\d\d)', expand=True)
clean_wines

,country,description,designation,points,price,province,region_1,taster_name,title,variety,winery,vintage
3,US,"Pineapple rind, lemon pith and orange blossom ...",Reserve Late Harvest,87,13.0,Michigan,Lake Michigan Shore,Alexander Peartree,St. Julian 2013 Reserve Late Harvest Riesling ...,Riesling,St. Julian,2013
4,US,"Much like the regular bottling from 2012, this...",Vintner's Reserve Wild Child Block,87,65.0,Oregon,Willamette Valley,Paul Gregutt,Sweet Cheeks 2012 Vintner's Reserve Wild Child...,Pinot Noir,Sweet Cheeks,2012
5,Spain,Blackberry and raspberry aromas show a typical...,Ars In Vitro,87,15.0,Northern Spain,Navarra,Michael Schachner,Tandem 2011 Ars In Vitro Tempranillo-Merlot (N...,Tempranillo-Merlot,Tandem,2011
6,Italy,"Here's a bright, informal red that opens with ...",Belsito,87,16.0,Sicily & Sardinia,Vittoria,Kerin O’Keefe,Terre di Giurfo 2013 Belsito Frappato (Vittoria),Frappato,Terre di Giurfo,2013
9,France,This has great depth of flavor with its fresh ...,Les Natures,87,27.0,Alsace,Alsace,Roger Voss,Jean-Baptiste Adam 2012 Les Natures Pinot Gris...,Pinot Gris,Jean-Baptiste Adam,2012
...,...,...,...,...,...,...,...,...,...,...,...,...
129959,France,The granite soil of the Brand Grand Cru vineya...,Brand Grand Cru,90,57.0,Alsace,Alsace,Roger Voss,Cave de Turckheim 2010 Brand Grand Cru Pinot G...,Pinot Gris,Cave de Turckheim,2010
129962,Italy,"Blackberry, cassis, grilled herb and toasted a...",Sàgana Tenuta San Giacomo,90,40.0,Sicily & Sardinia,Sicilia,Kerin O’Keefe,Cusumano 2012 Sàgana Tenuta San Giacomo Nero d...,Nero d'Avola,Cusumano,2012
129965,France,"While it's rich, this beautiful dry wine also ...",Seppi Landmann Vallée Noble,90,28.0,Alsace,Alsace,Roger Voss,Domaine Rieflé-Landmann 2013 Seppi Landmann Va...,Pinot Gris,Domaine Rieflé-Landmann,2013
129968,France,Well-drained gravel soil gives this wine its c...,Kritt,90,30.0,Alsace,Alsace,Roger Voss,Domaine Gresser 2013 Kritt Gewurztraminer (Als...,Gewürztraminer,Domaine Gresser,2013


In [6]:
clean_wines['type'] = ['white' if x in ['Chardonnay', 'Riesling', 'Sauvignon Blanc', 'White Blend', 'Sparkling Blend', 'Pinot Gris',
                                        'Champagne Blend', 'GrÃ¼ner Veltliner', 'Pinot Grigio', 'Portuguese White', 'Viognier',
                                        'GewÃ¼rztraminer', 'GewÃ¼rztraminer'] else 'red' for x in clean_wines['variety']]
clean_wines


,country,description,designation,points,price,province,region_1,taster_name,title,variety,winery,vintage,type
3,US,"Pineapple rind, lemon pith and orange blossom ...",Reserve Late Harvest,87,13.0,Michigan,Lake Michigan Shore,Alexander Peartree,St. Julian 2013 Reserve Late Harvest Riesling ...,Riesling,St. Julian,2013,white
4,US,"Much like the regular bottling from 2012, this...",Vintner's Reserve Wild Child Block,87,65.0,Oregon,Willamette Valley,Paul Gregutt,Sweet Cheeks 2012 Vintner's Reserve Wild Child...,Pinot Noir,Sweet Cheeks,2012,red
5,Spain,Blackberry and raspberry aromas show a typical...,Ars In Vitro,87,15.0,Northern Spain,Navarra,Michael Schachner,Tandem 2011 Ars In Vitro Tempranillo-Merlot (N...,Tempranillo-Merlot,Tandem,2011,red
6,Italy,"Here's a bright, informal red that opens with ...",Belsito,87,16.0,Sicily & Sardinia,Vittoria,Kerin O’Keefe,Terre di Giurfo 2013 Belsito Frappato (Vittoria),Frappato,Terre di Giurfo,2013,red
9,France,This has great depth of flavor with its fresh ...,Les Natures,87,27.0,Alsace,Alsace,Roger Voss,Jean-Baptiste Adam 2012 Les Natures Pinot Gris...,Pinot Gris,Jean-Baptiste Adam,2012,white
...,...,...,...,...,...,...,...,...,...,...,...,...,...
129959,France,The granite soil of the Brand Grand Cru vineya...,Brand Grand Cru,90,57.0,Alsace,Alsace,Roger Voss,Cave de Turckheim 2010 Brand Grand Cru Pinot G...,Pinot Gris,Cave de Turckheim,2010,white
129962,Italy,"Blackberry, cassis, grilled herb and toasted a...",Sàgana Tenuta San Giacomo,90,40.0,Sicily & Sardinia,Sicilia,Kerin O’Keefe,Cusumano 2012 Sàgana Tenuta San Giacomo Nero d...,Nero d'Avola,Cusumano,2012,red
129965,France,"While it's rich, this beautiful dry wine also ...",Seppi Landmann Vallée Noble,90,28.0,Alsace,Alsace,Roger Voss,Domaine Rieflé-Landmann 2013 Seppi Landmann Va...,Pinot Gris,Domaine Rieflé-Landmann,2013,white
129968,France,Well-drained gravel soil gives this wine its c...,Kritt,90,30.0,Alsace,Alsace,Roger Voss,Domaine Gresser 2013 Kritt Gewurztraminer (Als...,Gewürztraminer,Domaine Gresser,2013,red


In [7]:
clean_wines['word_count'] = clean_wines['description'].str.lower().str.split().str.len()
clean_wines

,country,description,designation,points,price,province,region_1,taster_name,title,variety,winery,vintage,type,word_count
3,US,"Pineapple rind, lemon pith and orange blossom ...",Reserve Late Harvest,87,13.0,Michigan,Lake Michigan Shore,Alexander Peartree,St. Julian 2013 Reserve Late Harvest Riesling ...,Riesling,St. Julian,2013,white,33
4,US,"Much like the regular bottling from 2012, this...",Vintner's Reserve Wild Child Block,87,65.0,Oregon,Willamette Valley,Paul Gregutt,Sweet Cheeks 2012 Vintner's Reserve Wild Child...,Pinot Noir,Sweet Cheeks,2012,red,41
5,Spain,Blackberry and raspberry aromas show a typical...,Ars In Vitro,87,15.0,Northern Spain,Navarra,Michael Schachner,Tandem 2011 Ars In Vitro Tempranillo-Merlot (N...,Tempranillo-Merlot,Tandem,2011,red,42
6,Italy,"Here's a bright, informal red that opens with ...",Belsito,87,16.0,Sicily & Sardinia,Vittoria,Kerin O’Keefe,Terre di Giurfo 2013 Belsito Frappato (Vittoria),Frappato,Terre di Giurfo,2013,red,31
9,France,This has great depth of flavor with its fresh ...,Les Natures,87,27.0,Alsace,Alsace,Roger Voss,Jean-Baptiste Adam 2012 Les Natures Pinot Gris...,Pinot Gris,Jean-Baptiste Adam,2012,white,30
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
129959,France,The granite soil of the Brand Grand Cru vineya...,Brand Grand Cru,90,57.0,Alsace,Alsace,Roger Voss,Cave de Turckheim 2010 Brand Grand Cru Pinot G...,Pinot Gris,Cave de Turckheim,2010,white,52
129962,Italy,"Blackberry, cassis, grilled herb and toasted a...",Sàgana Tenuta San Giacomo,90,40.0,Sicily & Sardinia,Sicilia,Kerin O’Keefe,Cusumano 2012 Sàgana Tenuta San Giacomo Nero d...,Nero d'Avola,Cusumano,2012,red,37
129965,France,"While it's rich, this beautiful dry wine also ...",Seppi Landmann Vallée Noble,90,28.0,Alsace,Alsace,Roger Voss,Domaine Rieflé-Landmann 2013 Seppi Landmann Va...,Pinot Gris,Domaine Rieflé-Landmann,2013,white,32
129968,France,Well-drained gravel soil gives this wine its c...,Kritt,90,30.0,Alsace,Alsace,Roger Voss,Domaine Gresser 2013 Kritt Gewurztraminer (Als...,Gewürztraminer,Domaine Gresser,2013,red,43


In [8]:
clean_wines = clean_wines[clean_wines['vintage'].notna()]
clean_wines = clean_wines.copy()
clean_wines['vintage'] = clean_wines['vintage'].astype(int)
clean_wines['age'] = 2020 - clean_wines['vintage'] 
clean_wines

,country,description,designation,points,price,province,region_1,taster_name,title,variety,winery,vintage,type,word_count,age
3,US,"Pineapple rind, lemon pith and orange blossom ...",Reserve Late Harvest,87,13.0,Michigan,Lake Michigan Shore,Alexander Peartree,St. Julian 2013 Reserve Late Harvest Riesling ...,Riesling,St. Julian,2013,white,33,7
4,US,"Much like the regular bottling from 2012, this...",Vintner's Reserve Wild Child Block,87,65.0,Oregon,Willamette Valley,Paul Gregutt,Sweet Cheeks 2012 Vintner's Reserve Wild Child...,Pinot Noir,Sweet Cheeks,2012,red,41,8
5,Spain,Blackberry and raspberry aromas show a typical...,Ars In Vitro,87,15.0,Northern Spain,Navarra,Michael Schachner,Tandem 2011 Ars In Vitro Tempranillo-Merlot (N...,Tempranillo-Merlot,Tandem,2011,red,42,9
6,Italy,"Here's a bright, informal red that opens with ...",Belsito,87,16.0,Sicily & Sardinia,Vittoria,Kerin O’Keefe,Terre di Giurfo 2013 Belsito Frappato (Vittoria),Frappato,Terre di Giurfo,2013,red,31,7
9,France,This has great depth of flavor with its fresh ...,Les Natures,87,27.0,Alsace,Alsace,Roger Voss,Jean-Baptiste Adam 2012 Les Natures Pinot Gris...,Pinot Gris,Jean-Baptiste Adam,2012,white,30,8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
129959,France,The granite soil of the Brand Grand Cru vineya...,Brand Grand Cru,90,57.0,Alsace,Alsace,Roger Voss,Cave de Turckheim 2010 Brand Grand Cru Pinot G...,Pinot Gris,Cave de Turckheim,2010,white,52,10
129962,Italy,"Blackberry, cassis, grilled herb and toasted a...",Sàgana Tenuta San Giacomo,90,40.0,Sicily & Sardinia,Sicilia,Kerin O’Keefe,Cusumano 2012 Sàgana Tenuta San Giacomo Nero d...,Nero d'Avola,Cusumano,2012,red,37,8
129965,France,"While it's rich, this beautiful dry wine also ...",Seppi Landmann Vallée Noble,90,28.0,Alsace,Alsace,Roger Voss,Domaine Rieflé-Landmann 2013 Seppi Landmann Va...,Pinot Gris,Domaine Rieflé-Landmann,2013,white,32,7
129968,France,Well-drained gravel soil gives this wine its c...,Kritt,90,30.0,Alsace,Alsace,Roger Voss,Domaine Gresser 2013 Kritt Gewurztraminer (Als...,Gewürztraminer,Domaine Gresser,2013,red,43,7


In [9]:
clean_wines = clean_wines.drop_duplicates('description')
clean_wines = clean_wines[pd.notnull(clean_wines.price)]

In [10]:
clean_wines.quantile(q=0.75, axis=0, numeric_only=True)

points          91.0
price           49.0
vintage       2014.0
word_count      49.0
age             10.0
Name: 0.75, dtype: float64

In [11]:
clean_wines['good_meh'] = clean_wines['points'].apply(lambda x: 'good' if x >= 91 else 'meh')
clean_wines

,country,description,designation,points,price,province,region_1,taster_name,title,variety,winery,vintage,type,word_count,age,good_meh
3,US,"Pineapple rind, lemon pith and orange blossom ...",Reserve Late Harvest,87,13.0,Michigan,Lake Michigan Shore,Alexander Peartree,St. Julian 2013 Reserve Late Harvest Riesling ...,Riesling,St. Julian,2013,white,33,7,meh
4,US,"Much like the regular bottling from 2012, this...",Vintner's Reserve Wild Child Block,87,65.0,Oregon,Willamette Valley,Paul Gregutt,Sweet Cheeks 2012 Vintner's Reserve Wild Child...,Pinot Noir,Sweet Cheeks,2012,red,41,8,meh
5,Spain,Blackberry and raspberry aromas show a typical...,Ars In Vitro,87,15.0,Northern Spain,Navarra,Michael Schachner,Tandem 2011 Ars In Vitro Tempranillo-Merlot (N...,Tempranillo-Merlot,Tandem,2011,red,42,9,meh
6,Italy,"Here's a bright, informal red that opens with ...",Belsito,87,16.0,Sicily & Sardinia,Vittoria,Kerin O’Keefe,Terre di Giurfo 2013 Belsito Frappato (Vittoria),Frappato,Terre di Giurfo,2013,red,31,7,meh
9,France,This has great depth of flavor with its fresh ...,Les Natures,87,27.0,Alsace,Alsace,Roger Voss,Jean-Baptiste Adam 2012 Les Natures Pinot Gris...,Pinot Gris,Jean-Baptiste Adam,2012,white,30,8,meh
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
129959,France,The granite soil of the Brand Grand Cru vineya...,Brand Grand Cru,90,57.0,Alsace,Alsace,Roger Voss,Cave de Turckheim 2010 Brand Grand Cru Pinot G...,Pinot Gris,Cave de Turckheim,2010,white,52,10,meh
129962,Italy,"Blackberry, cassis, grilled herb and toasted a...",Sàgana Tenuta San Giacomo,90,40.0,Sicily & Sardinia,Sicilia,Kerin O’Keefe,Cusumano 2012 Sàgana Tenuta San Giacomo Nero d...,Nero d'Avola,Cusumano,2012,red,37,8,meh
129965,France,"While it's rich, this beautiful dry wine also ...",Seppi Landmann Vallée Noble,90,28.0,Alsace,Alsace,Roger Voss,Domaine Rieflé-Landmann 2013 Seppi Landmann Va...,Pinot Gris,Domaine Rieflé-Landmann,2013,white,32,7,meh
129968,France,Well-drained gravel soil gives this wine its c...,Kritt,90,30.0,Alsace,Alsace,Roger Voss,Domaine Gresser 2013 Kritt Gewurztraminer (Als...,Gewürztraminer,Domaine Gresser,2013,red,43,7,meh


In [12]:
good_wines = clean_wines[clean_wines['good_meh'] == 'good']
good_wines

,country,description,designation,points,price,province,region_1,taster_name,title,variety,winery,vintage,type,word_count,age,good_meh
123,Australia,The blend is roughly two-thirds Shiraz and one...,Parson's Flat,92,40.0,South Australia,Padthaway,Joe Czerwinski,Henry's Drive Vignerons 2006 Parson's Flat Shi...,Shiraz-Cabernet Sauvignon,Henry's Drive Vignerons,2006,red,36,14,good
144,US,"Thick and brooding, this dark, sweetly tannic ...",K Block,91,85.0,California,Spring Mountain District,Virginie Boone,Terra Valentine 2013 K Block Cabernet Sauvigno...,Cabernet Sauvignon,Terra Valentine,2013,red,37,7,good
145,US,"Cooked cranberry is spiced with anise, pepperc...",Guidotti Vineyard,91,64.0,California,Santa Lucia Highlands,Matt Kettmann,Testarossa 2013 Guidotti Vineyard Pinot Noir (...,Pinot Noir,Testarossa,2013,red,43,7,good
146,US,"There's a touch of hot asphalt, pencil lead an...",Family Reserve,91,68.0,California,Santa Ynez Valley,Matt Kettmann,Vincent Vineyards 2010 Family Reserve Cabernet...,Cabernet Sauvignon,Vincent Vineyards,2010,red,51,10,good
147,US,"Black cherry, black plum and black currant are...",Family Reserve,91,68.0,California,Santa Ynez Valley,Matt Kettmann,Vincent Vineyards 2012 Family Reserve Cabernet...,Cabernet Sauvignon,Vincent Vineyards,2012,red,55,8,good
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
129928,Spain,This represents Torres' highest effort in Prio...,Salmos Perpetual,91,52.0,Catalonia,Priorat,Michael Schachner,Torres 2005 Salmos Perpetual Red (Priorat),Red Blend,Torres,2005,red,47,15,good
129930,Spain,"Arguably on the heavy and rich side, but if yo...",Crianza,91,26.0,Northern Spain,Ribera del Duero,Michael Schachner,Costaval 2005 Crianza (Ribera del Duero),Tempranillo Blend,Costaval,2005,red,48,15,good
129932,Argentina,Andeluna's top wines tend to be ripe and plump...,Pasionado,91,55.0,Mendoza Province,Uco Valley,Michael Schachner,Andeluna 2004 Pasionado Red (Uco Valley),Red Blend,Andeluna,2004,red,61,16,good
129936,France,A blend from several premier cru vineyards in ...,Premier Cru,91,66.0,Burgundy,Pommard,Roger Voss,Bouchard Père & Fils 2005 Premier Cru (Pommard),Pinot Noir,Bouchard Père & Fils,2005,red,61,15,good


In [13]:
clean_wines['char_count'] = clean_wines["description"].apply(lambda x: sum(len(word) for word in str(x).split(" ")))
clean_wines['sentence_count'] = clean_wines["description"].apply(lambda x: len(str(x).split(".")))
clean_wines['avg_word_length'] = clean_wines['char_count'] / clean_wines['word_count']
clean_wines['avg_sentence_lenght'] = clean_wines['word_count'] / clean_wines['sentence_count']
clean_wines

,country,description,designation,points,price,province,region_1,taster_name,title,variety,winery,vintage,type,word_count,age,good_meh,char_count,sentence_count,avg_word_length,avg_sentence_lenght
3,US,"Pineapple rind, lemon pith and orange blossom ...",Reserve Late Harvest,87,13.0,Michigan,Lake Michigan Shore,Alexander Peartree,St. Julian 2013 Reserve Late Harvest Riesling ...,Riesling,St. Julian,2013,white,33,7,meh,167,3,5.060606,11.000000
4,US,"Much like the regular bottling from 2012, this...",Vintner's Reserve Wild Child Block,87,65.0,Oregon,Willamette Valley,Paul Gregutt,Sweet Cheeks 2012 Vintner's Reserve Wild Child...,Pinot Noir,Sweet Cheeks,2012,red,41,8,meh,209,3,5.097561,13.666667
5,Spain,Blackberry and raspberry aromas show a typical...,Ars In Vitro,87,15.0,Northern Spain,Navarra,Michael Schachner,Tandem 2011 Ars In Vitro Tempranillo-Merlot (N...,Tempranillo-Merlot,Tandem,2011,red,42,9,meh,220,4,5.238095,10.500000
6,Italy,"Here's a bright, informal red that opens with ...",Belsito,87,16.0,Sicily & Sardinia,Vittoria,Kerin O’Keefe,Terre di Giurfo 2013 Belsito Frappato (Vittoria),Frappato,Terre di Giurfo,2013,red,31,7,meh,151,3,4.870968,10.333333
9,France,This has great depth of flavor with its fresh ...,Les Natures,87,27.0,Alsace,Alsace,Roger Voss,Jean-Baptiste Adam 2012 Les Natures Pinot Gris...,Pinot Gris,Jean-Baptiste Adam,2012,white,30,8,meh,131,4,4.366667,7.500000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
129959,France,The granite soil of the Brand Grand Cru vineya...,Brand Grand Cru,90,57.0,Alsace,Alsace,Roger Voss,Cave de Turckheim 2010 Brand Grand Cru Pinot G...,Pinot Gris,Cave de Turckheim,2010,white,52,10,meh,237,5,4.557692,10.400000
129962,Italy,"Blackberry, cassis, grilled herb and toasted a...",Sàgana Tenuta San Giacomo,90,40.0,Sicily & Sardinia,Sicilia,Kerin O’Keefe,Cusumano 2012 Sàgana Tenuta San Giacomo Nero d...,Nero d'Avola,Cusumano,2012,red,37,8,meh,184,4,4.972973,9.250000
129965,France,"While it's rich, this beautiful dry wine also ...",Seppi Landmann Vallée Noble,90,28.0,Alsace,Alsace,Roger Voss,Domaine Rieflé-Landmann 2013 Seppi Landmann Va...,Pinot Gris,Domaine Rieflé-Landmann,2013,white,32,7,meh,180,3,5.625000,10.666667
129968,France,Well-drained gravel soil gives this wine its c...,Kritt,90,30.0,Alsace,Alsace,Roger Voss,Domaine Gresser 2013 Kritt Gewurztraminer (Als...,Gewürztraminer,Domaine Gresser,2013,red,43,7,meh,183,4,4.255814,10.750000


In [14]:
winery_counts = {'winery' : clean_wines['winery'].unique(), 
                 'winery_count' : clean_wines['winery'].value_counts() 

            }
winery_df = pd.DataFrame(data=winery_counts)
winery_df.reset_index(drop=True, inplace=True)
winery_df

,winery,winery_count
0,St. Julian,144
1,Sweet Cheeks,117
2,Tandem,103
3,Terre di Giurfo,100
4,Jean-Baptiste Adam,90
...,...,...
8286,Yacochuya,1
8287,Tollot Beaut et Fils,1
8288,Bottega,1
8289,Bodegas Eidosela,1


In [15]:
clean_wines = clean_wines.merge(winery_df, how='inner', on='winery')
clean_wines.sample(30)

,country,description,designation,points,price,province,region_1,taster_name,title,variety,...,vintage,type,word_count,age,good_meh,char_count,sentence_count,avg_word_length,avg_sentence_lenght,winery_count
44071,Italy,"Enticing scents of Spanish broom, citrus bloss...",Fiore,91,26.0,Tuscany,Vernaccia di San Gimignano,Kerin O’Keefe,Montenidoli 2014 Fiore (Vernaccia di San Gimi...,Vernaccia,...,2014,red,55,6,good,309,4,5.618182,13.750000,1
42669,France,Toast and ripe spice go well with the warm red...,Médeville Collection,90,42.0,Bordeaux,Cadillac Côtes de Bordeaux,Roger Voss,Château Fayau 2014 Médeville Collection (Cadi...,Bordeaux-style Red Blend,...,2014,red,53,6,meh,237,5,4.471698,10.600000,2
25767,Spain,"Coffee, mocha, char and roasted black-fruit ar...",Mirto,89,70.0,Northern Spain,Rioja,Michael Schachner,Ramón Bilbao 2008 Mirto (Rioja),Tempranillo,...,2008,red,46,12,meh,248,5,5.391304,9.200000,6
6947,US,Deep and dark blueberry aromas are laced with ...,AUX,92,65.0,California,Paso Robles,Matt Kettmann,Daou 2012 AUX Red (Paso Robles),Rhône-style Red Blend,...,2012,red,51,8,good,255,3,5.000000,17.000000,21
21915,US,"Picturesque with a crystalline salmon color, i...",Dianthus Estate Mourvèdre-Grenache-Counoise,87,27.0,California,Paso Robles,Matt Kettmann,Tablas Creek 2013 Dianthus Estate Mourvèdre-Gr...,Rosé,...,2013,red,65,7,meh,343,5,5.276923,13.000000,8
28473,Italy,Delicate scents of apple and white flower lead...,Punggl,88,17.0,Northeastern Italy,Alto Adige,Kerin O’Keefe,Peter Zemmer 2014 Punggl Pinot Bianco (Alto Ad...,Pinot Bianco,...,2014,red,35,6,meh,152,3,4.342857,11.666667,5
2863,Spain,"Tobacco, dill and vanilla aromas transition to...",Coto Real Reserva,86,45.0,Northern Spain,Rioja,Michael Schachner,El Coto 2008 Coto Real Reserva (Rioja),Tempranillo,...,2008,red,36,12,meh,204,4,5.666667,9.000000,33
45692,France,"This blend of Grenache Blanc, Grenache Gris, R...",Carrément Blanc,86,43.0,Languedoc-Roussillon,Côtes du Roussillon,Lauren Buzzeo,Domaine de l'Edre 2010 Carrément Blanc White (...,Rhône-style White Blend,...,2010,red,57,10,meh,276,3,4.842105,19.000000,1
18769,US,"Structured in firm tannins, this seamless wine...",First Born,93,58.0,California,Sonoma Coast,Virginie Boone,Gros Ventre 2012 First Born Pinot Noir (Sonoma...,Pinot Noir,...,2012,red,61,8,good,341,4,5.590164,15.250000,9
25056,US,"Aromas of barrel spice, dried green herb, drie...",96 Cedars,87,25.0,Washington,Columbia Valley (WA),Sean P. Sullivan,Auclair 2014 96 Cedars Red (Columbia Valley (WA)),Bordeaux-style Red Blend,...,2014,red,29,6,meh,133,3,4.586207,9.666667,6


In [16]:
country_counts = {'country' : clean_wines['country'].unique(),
             'country_count' : clean_wines['country'].value_counts()
            }
country_df = pd.DataFrame(data=country_counts)
country_df.reset_index(drop=True, inplace=True)
country_df

,country,country_count
0,US,23237
1,Spain,9408
2,Italy,6120
3,France,4337
4,Argentina,2579
5,Australia,1309
6,Canada,154


In [17]:
clean_wines = clean_wines.merge(country_df, how='inner', on='country')
clean_wines.sample(30)

,country,description,designation,points,price,province,region_1,taster_name,title,variety,...,type,word_count,age,good_meh,char_count,sentence_count,avg_word_length,avg_sentence_lenght,winery_count,country_count
28853,Italy,"This brut offers notes of citrus, stone fruit ...",Brut Vintage,88,35.0,Lombardy,Franciacorta,Kerin O’Keefe,Fratelli Berlucchi 2009 Brut Vintage Sparkling...,Sparkling Blend,...,white,29,11,meh,146,4,5.034483,7.250000,11,6120
39280,France,"Rich and round, the wine boasts ripe toast and...",Les Trois Pêcheurs,88,18.0,Burgundy,Saint-Véran,Roger Voss,Vignerons des Terres Secrètes 2008 Les Trois P...,Chardonnay,...,white,26,12,meh,127,3,4.884615,8.666667,5,4337
11213,US,"From Northridge Vineyard grapes, this mix of f...",The Hidden,97,70.0,Washington,Wahluke Slope,Paul Gregutt,K Vintners 2009 The Hidden Syrah (Wahluke Slope),Syrah,...,red,62,11,good,324,5,5.225806,12.400000,9,23237
42923,France,"This textured, ripe wine shows the potential o...",Roche Noire,90,35.0,Beaujolais,Beaujolais Blanc,Roger Voss,Domaine Lathuilière-Gravallon 2014 Roche Noire...,Chardonnay,...,white,54,6,meh,252,6,4.666667,9.000000,1,4337
3148,US,This is a blend of two-thirds Merlot topped of...,Le Coursier,91,34.0,Washington,Columbia Valley (WA),Sean P. Sullivan,Brian Carter Cellars 2011 Le Coursier Red (Col...,Bordeaux-style Red Blend,...,red,38,9,good,204,4,5.368421,9.500000,25,23237
12001,US,"A thin wine, with a quick strawberry/cranberry...",Ruby Vineyard,84,34.0,Oregon,Willamette Valley,Paul Gregutt,Apolloni 2008 Ruby Vineyard Pinot Noir (Willam...,Pinot Noir,...,red,33,12,meh,171,3,5.181818,11.000000,8,23237
20991,US,"This wine is principally Syrah, Cabernet Sauvi...",Alluvial,88,20.0,Washington,Columbia Valley (WA),Sean P. Sullivan,Gravel Bar 2013 Alluvial Red (Columbia Valley ...,Red Blend,...,red,34,7,meh,196,3,5.764706,11.333333,2,23237
19961,US,Scents of smoke and ash dominate and obscure t...,Bench Lands,86,22.0,Oregon,Oregon,Paul Gregutt,Brandborg 2008 Bench Lands Pinot Noir (Oregon),Pinot Noir,...,red,62,12,meh,296,4,4.774194,15.500000,3,23237
37687,France,Gascony was the birthplace of the Alexandre Du...,Terres d'Artagnan Cuvée Excellence,86,7.0,Southwest France,Côtes de Gascogne,Roger Voss,Producteurs Plaimont 2013 Terres d'Artagnan Cu...,White Blend,...,white,35,7,meh,185,4,5.285714,8.750000,7,4337
9980,US,This wine is subtle in aroma but extremely gen...,Retrofit,91,20.0,California,North Coast,Jim Gordon,Replica 2015 Retrofit Chardonnay (North Coast),Chardonnay,...,white,41,5,good,187,3,4.560976,13.666667,10,23237


In [18]:
province_counts = {'province' : clean_wines['province'].unique(),
             'province_count' : clean_wines['province'].value_counts()
            }
province_df = pd.DataFrame(data=province_counts)
province_df.reset_index(drop=True, inplace=True)
province_df

,province,province_count
0,Michigan,12491
1,Oregon,5325
2,California,3423
3,Washington,2770
4,Virginia,2209
5,New York,1928
6,Idaho,1824
7,Texas,1574
8,Pennsylvania,1462
9,New Jersey,1383


In [19]:
clean_wines = clean_wines.merge(province_df, how='inner', on='province')
clean_wines.sample(30)

,country,description,designation,points,price,province,region_1,taster_name,title,variety,...,word_count,age,good_meh,char_count,sentence_count,avg_word_length,avg_sentence_lenght,winery_count,country_count,province_count
20273,US,"A white Bordeaux-style blend, crisp and edgy, ...",Updraft,86,17.0,Washington,Columbia Valley (WA),Paul Gregutt,Cloudlift Cellars 2010 Updraft White (Columbia...,Bordeaux-style White Blend,...,27,10,meh,136,3,5.037037,9.000000,5,23237,2770
231,US,"Crisply defined, this cuvée, from a vineyard w...",Tualatin Estate,88,45.0,Oregon,Willamette Valley,Paul Gregutt,Willamette Valley Vineyards 2010 Tualatin Esta...,Pinot Noir,...,34,10,meh,162,3,4.764706,11.333333,40,23237,5325
44572,Argentina,"Clean, fresh, stony aromas are basic but good....",Estate,87,10.0,Mendoza Province,Mendoza,Michael Schachner,Finca El Origen 2015 Estate Chardonnay (Mendoza),Chardonnay,...,26,5,meh,140,3,5.384615,8.666667,7,2579,9
29231,Italy,"Aromas of tobacco, dried rose, mature berry an...",Riserva,92,70.0,Tuscany,Brunello di Montalcino,Kerin O’Keefe,Marchesato degli Aleramici 2010 Riserva (Brun...,Sangiovese,...,45,10,good,209,3,4.644444,15.000000,3,6120,254
40317,France,"While it's firmly tannic, the fruit still show...",B (Si),88,16.0,Southwest France,Cahors,Roger Voss,Lannac Saint-Jean 2009 B (Si) Malbec (Cahors),Malbec,...,28,11,meh,138,3,4.928571,9.333333,2,4337,17
31426,Italy,Fruity aromas of mature berry mingle with note...,Costa Bruna,90,25.0,Piedmont,Barbera d'Alba,Kerin O’Keefe,Poderi Colla 2013 Costa Bruna (Barbera d'Alba),Barbera,...,41,7,meh,223,3,5.439024,13.666667,28,6120,154
27212,Spain,Ripe roasted black-plum and berry aromas are w...,Joven,85,10.0,Levante,Jumilla,Michael Schachner,Casa de la Ermita 2015 Joven Monastrell-Syrah ...,Monastrell-Syrah,...,46,5,meh,217,5,4.717391,9.200000,2,9408,381
28729,Italy,"Purple flower, dark-skinned berry and brown sp...",Salterio,88,23.0,Tuscany,Rosso di Montepulciano,Kerin O’Keefe,Trerose 2015 Salterio (Rosso di Montepulciano),Red Blend,...,34,5,meh,179,4,5.264706,8.500000,6,6120,254
34634,France,The granite soil of the Brand Grand Cru vineya...,Brand Grand Cru,90,57.0,Alsace,Alsace,Roger Voss,Cave de Turckheim 2010 Brand Grand Cru Pinot G...,Pinot Gris,...,52,10,meh,237,5,4.557692,10.400000,5,4337,57
31478,Italy,"Rose, chopped mint, red berry and new leather ...",Coste di Rose,92,85.0,Piedmont,Barolo,Kerin O’Keefe,Marchesi di Barolo 2013 Coste di Rose (Barolo),Nebbiolo,...,45,7,good,249,5,5.533333,9.000000,26,6120,154


In [20]:
region_counts = {'region_1' : clean_wines['region_1'].unique(),
             'region_count' : clean_wines['region_1'].value_counts()
            }
region_df = pd.DataFrame(data=region_counts)
region_df.reset_index(drop=True, inplace=True)
region_df

,region_1,region_count
0,Lake Michigan Shore,2240
1,Michigan,1509
2,Old Mission Peninsula,1483
3,Leelanau Peninsula,1344
4,Willamette Valley,1178
...,...,...
982,Short Hills Bench,1
983,St. David's Bench,1
984,Prince Edward County,1
985,Okanagan Valley,1


In [21]:
clean_wines = clean_wines.merge(region_df, how='inner', on='region_1')
clean_wines.sample(30)

,country,description,designation,points,price,province,region_1,taster_name,title,variety,...,age,good_meh,char_count,sentence_count,avg_word_length,avg_sentence_lenght,winery_count,country_count,province_count,region_count
18170,US,"A 50-50 Cabernet/Syrah blend, this is heavily ...",Confluence Red Blend,83,25.0,Washington,Columbia Valley (WA),Paul Gregutt,Cedar River Cellars 2010 Confluence Red Blend ...,Red Blend,...,10,meh,153,4,5.100000,7.500000,1,23237,2770,50
14077,US,"A succulent, almost thick mouthfeel and subtle...",Silvaspoons Vineyard,89,17.0,California,Lodi,Jim Gordon,Fenestra 2015 Silvaspoons Vineyard Verdelho (L...,Verdelho,...,5,meh,174,3,4.702703,12.333333,33,23237,3423,119
23735,Spain,"A really nice, dark, modern style of Rioja fro...",Amaren,92,50.0,Northern Spain,Rioja,Michael Schachner,Luis Cañas 2004 Amaren (Rioja),Tempranillo,...,16,good,280,6,4.827586,9.666667,9,9408,531,36
39434,France,"This plump, medium-bodied rosé is a blend of t...",Cape Bleue,86,13.0,France Other,Vin de Pays de la Méditerranée,Joe Czerwinski,Jean-Luc Colombo 2014 Cape Bleue Rosé (Vin de ...,Rosé,...,6,meh,167,3,5.387097,10.333333,12,4337,18,2
625,US,An elegant blend from different estate vineyar...,Premier Cuvée,91,54.0,Oregon,Willamette Valley,Paul Gregutt,Archery Summit 2013 Premier Cuvée Pinot Noir (...,Pinot Noir,...,7,good,190,3,5.277778,12.000000,12,23237,5325,1178
35747,France,"Soft and rounded, this has apple and peach fru...",La Sereine,88,20.0,Burgundy,Chablis,Roger Voss,La Chablisienne 2009 La Sereine (Chablis),Chardonnay,...,11,meh,121,3,5.041667,8.000000,45,4337,48,5
11793,US,"Plump aromas of black cherry, mulberry, vanill...",Single Vineyard La Coupelle,95,60.0,California,Arroyo Grande Valley,Matt Kettmann,Laetitia 2015 Single Vineyard La Coupelle Pino...,Pinot Noir,...,5,good,272,4,4.610169,14.750000,4,23237,3423,180
2123,US,"Formerly the Goldschmidt vineyard, Winderlea i...",Estate,91,60.0,Oregon,Dundee Hills,Paul Gregutt,Winderlea 2007 Estate Pinot Noir (Dundee Hills),Pinot Noir,...,13,good,259,4,5.755556,11.250000,37,23237,5325,713
31514,Italy,This Barolo shows funky aromas of rubber and a...,Gabutti Bussia,84,75.0,Piedmont,Barolo,Kerin O’Keefe,Stroppiana 2009 Gabutti Bussia (Barolo),Nebbiolo,...,11,meh,203,4,4.951220,10.250000,16,6120,154,9
33573,Italy,"A blend of 80% Chardonnay and 20% Pinot Nero, ...",Brut Nature,87,45.0,Lombardy,Franciacorta,Kerin O’Keefe,Ronco Calino 2009 Brut Nature Sparkling (Franc...,Sparkling Blend,...,11,meh,181,3,5.027778,12.000000,2,6120,80,6


In [22]:
taster_counts = {'taster_name' : clean_wines['taster_name'].unique(),
             'taster_count' : clean_wines['taster_name'].value_counts()
            }
taster_df = pd.DataFrame(data=taster_counts)
taster_df.reset_index(drop=True, inplace=True)
taster_df

,taster_name,taster_count
0,Alexander Peartree,7433
1,Anna Lee C. Iijima,7022
2,Susan Kostrzewa,6065
3,Paul Gregutt,5978
4,Michael Schachner,5915
5,Joe Czerwinski,4068
6,Sean P. Sullivan,3013
7,Virginie Boone,2261
8,Jim Gordon,2119
9,Matt Kettmann,1490


In [23]:
clean_wines = clean_wines.merge(taster_df, how='inner', on='taster_name')
clean_wines.sample(30)

,country,description,designation,points,price,province,region_1,taster_name,title,variety,...,good_meh,char_count,sentence_count,avg_word_length,avg_sentence_lenght,winery_count,country_count,province_count,region_count,taster_count
3543,US,This is assembled from ten different vineyards...,Quartet Red Wine,86,25.0,Oregon,Applegate Valley,Paul Gregutt,Paschal 2000 Quartet Red Wine Red (Applegate V...,Bordeaux-style Red Blend,...,meh,251,4,4.826923,13.000000,3,23237,5325,1037,5978
41672,Italy,"Made with 40% Cabernet Sauvignon, 25% Merlot, ...",Fufluna,87,20.0,Tuscany,Toscana,Kerin O’Keefe,Tenuta Poggio Rosso 2015 Fufluna Red (Toscana),Red Blend,...,meh,220,4,4.782609,11.500000,1,6120,254,19,84
15754,Australia,This is a well-priced example of Barossa Caber...,Grand Barossa,88,16.0,South Australia,Barossa,Joe Czerwinski,Château Tanunda 2012 Grand Barossa Cabernet Sa...,Cabernet Sauvignon,...,meh,233,4,5.065217,11.500000,7,1309,5,1,4068
20034,US,This wine made from Muscat Canelli grapes is s...,Estate Grown,86,25.0,California,Napa Valley,Virginie Boone,St. Supéry 2014 Estate Grown Moscato (Napa Val...,Moscato,...,meh,241,3,5.355556,15.000000,25,23237,3423,420,2261
33722,France,From one of the rare south-facing vineyards in...,Clos des Couchereaux Premier Cru,94,62.0,Burgundy,Beaune,Roger Voss,Louis Jadot 2013 Clos des Couchereaux Premier ...,Pinot Noir,...,good,239,5,4.979167,9.600000,18,4337,48,4,927
31705,US,Aromas of warm peach custard meet with baked y...,Claudia Cuvée Estate,88,35.0,California,Adelaida District,Matt Kettmann,Alta Colina 2014 Claudia Cuvée Estate Marsanne...,Marsanne,...,meh,204,3,4.975610,13.666667,47,23237,3423,165,1490
46169,Italy,"Aromas of Spanish broom, honeysuckle and a yea...",Brut Metodo Classico,87,20.0,Italy Other,Vino Spumante,Kerin O’Keefe,Murgo 2010 Brut Metodo Classico Nerello Mascal...,Nerello Mascalese,...,meh,189,3,5.400000,11.666667,9,6120,74,5,84
22098,US,"Somewhat muted, this estate-grown wine is most...",Estate Grown Reserve,86,40.0,California,Russian River Valley,Virginie Boone,Balverne 2013 Estate Grown Reserve Pinot Noir ...,Pinot Noir,...,meh,141,3,4.700000,10.000000,2,23237,3423,415,2261
19847,US,This blend of Cinsault (75%) and Mourvèdre alr...,CM2,83,23.0,Washington,Rattlesnake Hills,Sean P. Sullivan,Vortex 2010 CM2 Red (Rattlesnake Hills),Red Blend,...,meh,150,3,5.172414,9.666667,5,23237,2770,46,3013
45055,Italy,This earthy and spicy Barbera possesses an int...,Quass,89,28.0,Piedmont,Barbera d'Alba,Kerin O’Keefe,Pecchenino 2011 Quass (Barbera d'Alba),Barbera,...,meh,184,3,5.257143,11.666667,28,6120,154,9,84


In [ ]:
winery_scores = {'winery' : clean_wines['winery'].unique(), 
                 'winery_avg' : clean_winesclean_wines['winery'].value_counts() 

            }
winery_df = pd.DataFrame(data=winery_counts)
winery_df.reset_index(drop=True, inplace=True)
winery_df

In [24]:
features = clean_wines[['price', 'age', 'word_count', 'char_count', 'sentence_count', 'avg_word_length', 
                        'avg_sentence_lenght', 'winery_count', 'country_count', 'province_count', 'region_count',
                       'taster_count']]
features

,price,age,word_count,char_count,sentence_count,avg_word_length,avg_sentence_lenght,winery_count,country_count,province_count,region_count,taster_count
0,13.0,7,33,167,3,5.060606,11.000000,144,23237,12491,2240,7433
1,20.0,7,48,250,4,5.208333,12.000000,144,23237,12491,2240,7433
2,13.0,7,28,136,3,4.857143,9.333333,144,23237,12491,2240,7433
3,20.0,7,39,208,4,5.333333,9.750000,144,23237,12491,2240,7433
4,13.0,7,38,172,3,4.526316,12.666667,144,23237,12491,2240,7433
...,...,...,...,...,...,...,...,...,...,...,...,...
47139,28.0,8,32,186,4,5.812500,8.000000,2,4337,57,5,8
47140,27.0,7,39,222,4,5.692308,9.750000,2,4337,57,5,8
47141,25.0,5,49,256,4,5.224490,12.250000,1,4337,57,5,8
47142,40.0,116,48,233,5,4.854167,9.600000,1,4337,57,5,8


In [25]:
no_price = features.drop(columns='price')
no_price

,age,word_count,char_count,sentence_count,avg_word_length,avg_sentence_lenght,winery_count,country_count,province_count,region_count,taster_count
0,7,33,167,3,5.060606,11.000000,144,23237,12491,2240,7433
1,7,48,250,4,5.208333,12.000000,144,23237,12491,2240,7433
2,7,28,136,3,4.857143,9.333333,144,23237,12491,2240,7433
3,7,39,208,4,5.333333,9.750000,144,23237,12491,2240,7433
4,7,38,172,3,4.526316,12.666667,144,23237,12491,2240,7433
...,...,...,...,...,...,...,...,...,...,...,...
47139,8,32,186,4,5.812500,8.000000,2,4337,57,5,8
47140,7,39,222,4,5.692308,9.750000,2,4337,57,5,8
47141,5,49,256,4,5.224490,12.250000,1,4337,57,5,8
47142,116,48,233,5,4.854167,9.600000,1,4337,57,5,8


In [26]:
X = features
y = clean_wines['good_meh']

In [27]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from tensorflow.keras.utils import to_categorical

X_train, X_test, y_train, y_test = train_test_split(
    X, y, random_state=1)
X_scaler = MinMaxScaler().fit(X_train)
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)


# Step 1: Label-encode data set
label_encoder = LabelEncoder()
label_encoder.fit(y_train)
encoded_y_train = label_encoder.transform(y_train)
encoded_y_test = label_encoder.transform(y_test)

# Step 2: Convert encoded labels to one-hot-encoding
y_train_categorical = to_categorical(encoded_y_train)
y_test_categorical = to_categorical(encoded_y_test)

In [29]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

model = Sequential()
model.add(Dense(units=200, activation='relu', input_dim=12))
model.add(Dense(units=200, activation='relu'))
model.add(Dense(units=2, activation='softmax'))

In [30]:
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
model.fit(
    X_train_scaled,
    y_train_categorical,
    epochs=60,
    shuffle=True,
    verbose=0
)

In [31]:
model_loss, model_accuracy = model.evaluate(
    X_test_scaled, y_test_categorical, verbose=2)
print(
    f"Normal Neural Network - Loss: {model_loss}, Accuracy: {model_accuracy}")

11786/11786 - 1s - loss: 0.3937 - accuracy: 0.8138
Normal Neural Network - Loss: 0.3936509790716684, Accuracy: 0.8137620687484741


In [32]:
encoded_predictions = model.predict_classes(X_test_scaled[:])
prediction_labels = label_encoder.inverse_transform(encoded_predictions)

In [33]:
d = {'prediction' : prediction_labels, 
     'actual' : y_test}

In [34]:
predictions_df = pd.DataFrame(data=d)
predictions_df.sample(30)

,prediction,actual
29392,meh,meh
39443,meh,meh
6611,good,meh
29901,good,good
42911,meh,meh
9526,meh,meh
20752,meh,meh
33187,meh,meh
26818,good,good
14737,good,good


In [ ]:
X = no_price
y = clean_wines['good_meh']

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from tensorflow.keras.utils import to_categorical

X_train, X_test, y_train, y_test = train_test_split(
    X, y, random_state=1)
X_scaler = MinMaxScaler().fit(X_train)
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)


# Step 1: Label-encode data set
label_encoder = LabelEncoder()
label_encoder.fit(y_train)
encoded_y_train = label_encoder.transform(y_train)
encoded_y_test = label_encoder.transform(y_test)

# Step 2: Convert encoded labels to one-hot-encoding
y_train_categorical = to_categorical(encoded_y_train)
y_test_categorical = to_categorical(encoded_y_test)

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

model = Sequential()
model.add(Dense(units=200, activation='relu', input_dim=6))
model.add(Dense(units=200, activation='relu'))
model.add(Dense(units=2, activation='softmax'))

In [ ]:
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
model.fit(
    X_train_scaled,
    y_train_categorical,
    epochs=60,
    shuffle=True,
    verbose=0
)

In [ ]:
model_loss, model_accuracy = model.evaluate(
    X_test_scaled, y_test_categorical, verbose=2)
print(
    f"Normal Neural Network - Loss: {model_loss}, Accuracy: {model_accuracy}")

In [ ]:
encoded_predictions = model.predict_classes(X_test_scaled[:])
prediction_labels = label_encoder.inverse_transform(encoded_predictions)

In [ ]:
d = {'prediction' : prediction_labels, 
     'actual' : y_test}

In [ ]:
predictions_df = pd.DataFrame(data=d)
predictions_df.sample(30)

In [ ]:
features = clean_wines[['price', 'age', 'word_count', 'char_count', 'sentence_count', 'avg_word_length', 
                        'avg_sentence_lenght', 'winery_count', 'country_count', 'province_count', 'region_count',
                       'taster_count']]
features

In [ ]:
X = features
y = clean_wines['type']

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from tensorflow.keras.utils import to_categorical

X_train, X_test, y_train, y_test = train_test_split(
    X, y, random_state=1)
X_scaler = MinMaxScaler().fit(X_train)
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)


# Step 1: Label-encode data set
label_encoder = LabelEncoder()
label_encoder.fit(y_train)
encoded_y_train = label_encoder.transform(y_train)
encoded_y_test = label_encoder.transform(y_test)

# Step 2: Convert encoded labels to one-hot-encoding
y_train_categorical = to_categorical(encoded_y_train)
y_test_categorical = to_categorical(encoded_y_test)

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

model = Sequential()
model.add(Dense(units=200, activation='relu', input_dim=12))
model.add(Dense(units=200, activation='relu'))
model.add(Dense(units=2, activation='softmax'))

In [ ]:
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
model.fit(
    X_train_scaled,
    y_train_categorical,
    epochs=60,
    shuffle=True,
    verbose=0
)

In [ ]:
model_loss, model_accuracy = model.evaluate(
    X_test_scaled, y_test_categorical, verbose=2)
print(
    f"Normal Neural Network - Loss: {model_loss}, Accuracy: {model_accuracy}")

In [ ]:
encoded_predictions = model.predict_classes(X_test_scaled[:])
prediction_labels = label_encoder.inverse_transform(encoded_predictions)

In [ ]:
d = {'prediction' : prediction_labels, 
     'actual' : y_test}

In [ ]:
predictions_df = pd.DataFrame(data=d)
predictions_df.sample(30)

In [ ]:
white_prd = predictions_df[predictions_df['prediction'] == 'white']
white_prd

In [ ]:
red_prd = predictions_df[predictions_df['prediction'] == 'red']
red_prd

In [ ]:
white_test_wines = predictions_df[predictions_df['actual'] == 'white']
white_test_wines.

In [ ]:
features = clean_wines[['price', 'word_count', 'age', 'points']]
features

In [ ]:
X = features
y = clean_wines['variety']

In [ ]:
clean_wines['variety'].nunique()

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from tensorflow.keras.utils import to_categorical

X_train, X_test, y_train, y_test = train_test_split(
    X, y, random_state=1)
X_scaler = MinMaxScaler().fit(X_train)
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)


# Step 1: Label-encode data set
label_encoder = LabelEncoder()
label_encoder.fit(y_train)
encoded_y_train = label_encoder.transform(y_train)
encoded_y_test = label_encoder.transform(y_test)

# Step 2: Convert encoded labels to one-hot-encoding
y_train_categorical = to_categorical(encoded_y_train)
y_test_categorical = to_categorical(encoded_y_test)